In [5]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip
from pyspark.sql.functions import sum, col
from google.colab import files

builder = (SparkSession.builder.appName("DeltaLakeETL")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog"))

spark = configure_spark_with_delta_pip(builder).getOrCreate()


In [6]:
# Upload and Load DataFrames
print("Upload sales.csv, inventory.csv, and products.csv")
uploaded = files.upload()
sales_df = spark.read.option("header", True).csv("sales.csv")
inventory_df = spark.read.option("header", True).csv("inventory.csv")
products_df = spark.read.option("header", True).csv("products.csv")

# Upload new incremental files for ETL simulation
print("Upload sales_new.csv and inventory_new.csv for ETL increment")
uploaded = files.upload()
sales_new_df = spark.read.option("header", True).csv("sales_new.csv")
inventory_new_df = spark.read.option("header", True).csv("inventory_new.csv")
# Append increment
sales_df = sales_df.union(sales_new_df)
inventory_df = inventory_df.union(inventory_new_df)

Upload sales.csv, inventory.csv, and products.csv


Saving inventory_new.csv to inventory_new (1).csv
Upload sales_new.csv and inventory_new.csv for ETL increment


Saving inventory.csv to inventory (1).csv


In [7]:
# Save as Delta
sales_delta_path = "/content/delta/sales"
inventory_delta_path = "/content/delta/inventory"
products_delta_path = "/content/delta/products"

sales_df.write.format("delta").mode("overwrite").save(sales_delta_path)
inventory_df.write.format("delta").mode("overwrite").save(inventory_delta_path)
products_df.write.format("delta").mode("overwrite").save(products_delta_path)

# Save as CSV and Parquet
sales_df.write.csv("sales_output.csv", header=True, mode="overwrite")
inventory_df.write.csv("inventory_output.csv", header=True, mode="overwrite")
products_df.write.csv("products_output.csv", header=True, mode="overwrite")

sales_df.write.parquet("sales_output.parquet", mode="overwrite")
inventory_df.write.parquet("inventory_output.parquet", mode="overwrite")
products_df.write.parquet("products_output.parquet", mode="overwrite")


In [8]:
sales_df.createOrReplaceTempView("sales")
products_df.createOrReplaceTempView("products")

top_products = spark.sql("""
SELECT p.product_name, SUM(CAST(s.quantity_sold AS INT)) AS total_quantity_sold
FROM sales s
JOIN products p ON s.product_id = p.product_id
GROUP BY p.product_name
ORDER BY total_quantity_sold DESC
LIMIT 5
""")
print("Top 5 Best-Selling Products:")
top_products.show()


Top 5 Best-Selling Products:
+---------------+-------------------+
|   product_name|total_quantity_sold|
+---------------+-------------------+
| Wireless Mouse|                 37|
| Laptop Model A|                 26|
| Laptop Model B|                 17|
|Gaming Keyboard|                 16|
|   Office Chair|                 15|
+---------------+-------------------+

